<a href="https://colab.research.google.com/github/tpgus2603/DataMining/blob/main/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd ./drive/MyDrive/json_csv_files

/content/drive/MyDrive/json_csv_files


/content/drive/MyDrive/json_csv_files/amazon_review2


'/content/drive/MyDrive/json_csv_files/amazon_review2'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# 라이브러리 설치 (필요시)
!pip install pandas scikit-learn

# 라이브러리 임포트
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# 상품 데이터 파일 경로
products_file_path = './combined_data2.json'  # 실제 파일 경로로 변경하세요

# 상품 데이터 로드 (JSON Lines 형식)
products_df = pd.read_json(products_file_path, lines=True)

# 원하는 카테고리 목록
desired_categories = ['Beauty', 'Car', 'Fashion', 'Grocery_Gourmet_Food', 'Sports_Outdoors']

# 카테고리 필터링
filtered_products_df = products_df[products_df['category'].isin(desired_categories)].copy()

# 필요 없는 열 제거
filtered_products_df = filtered_products_df[['rating', 'title', 'text', 'helpful_vote', 'verified_purchase', 'product_title', 'category', 'parent_asin']]

print("Filtered Products DataFrame:")
print(filtered_products_df.head())

Filtered Products DataFrame:
      rating                      title  \
5321       5                 Excellent!   
5322       5         Perfect repair kit   
5323       5           Awesome leathers   
5324       4  Nice product, sizing off!   
5325       4             Mixed feelings   

                                                   text  helpful_vote  \
5321                                 I love it. Pretty!             0   
5322  I have a great Weaver halter. Recently, the Ch...             0   
5323  These are amazing. My trainer noticed right aw...             6   
5324  Product is well made, but sizing seems way off...             1   
5325  I loved this chalk bag online, but in person, ...             0   

      verified_purchase                                      product_title  \
5321                  1             CamelBak eddy Glass Water Bottle, 24oz   
5322                  1     Weaver Leather Breakaway Fuse (3-Pack) , Brown   
5323                  0  Total Saddle F

In [18]:
# 사용자 리뷰 데이터 파일 경로
user_reviews_file_path = './user_reviews.json'  # 실제 파일 경로로 변경하세요

# 사용자 리뷰 데이터 로드 (JSON Lines 형식)
user_reviews_df = pd.read_json(user_reviews_file_path, lines=True)

# 필요 없는 열 제거
user_reviews_df = user_reviews_df[['title', 'text', 'helpful_vote', 'verified_purchase', 'product_title', 'category', 'parent_asin']]

# 원하는 카테고리만 필터링 (상품 데이터와 일치시킴)
user_reviews_df = user_reviews_df[user_reviews_df['category'].isin(desired_categories)].copy()

print("User Reviews DataFrame:")
print(user_reviews_df.head())

User Reviews DataFrame:
                                           title  \
0                                   clip rollers   
1  They smell good. They are just the right size   
2                       replacement filing discs   
3                           halo hair extensions   
4           30ft 50Amp heavy duty extension cord   

                                                text  helpful_vote  \
0  I purchased these to see if I could set my mom...             0   
1  I'm a BIG wet nap fan.  Always have been.  The...             4   
2  I purchased the tool for my 80 year old mom be...             1   
3  This halo hair extension is simply put, garbag...             4   
4  30 ft 50Amp heavy duty extension cord - husban...             0   

   verified_purchase                                      product_title  \
0                  1             1/8 inch Snap On Rollers 8 Pack (Pink)   
1                  1             Wet-Nap Moist Towelette (case of 1000)   
2                

In [19]:
def combine_text(df):
    return df['title'].fillna('') + ' ' + df['text'].fillna('') + ' ' + df['product_title'].fillna('')

# 평가 기준 설정 (rating == 5.0)
rating_threshold = 5.0

In [23]:
def get_recommendations_based_on_high_rating(user_reviews_df, high_rating_products_df, tfidf_matrix, asin_to_index, category_df, top_n=5):
    recommendations = []
    reviewed_asins = set(user_reviews_df['parent_asin'])
    for _, review in user_reviews_df.iterrows():
        asin = review['parent_asin']
        if asin not in asin_to_index:
            continue
        idx = asin_to_index[asin]
        cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
        similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

        similar_products_all = category_df.iloc[similar_indices]
        similar_products = similar_products_all[similar_products_all['rating'] == 5.0]
        recommendations.append(similar_products)

    if recommendations:
        recommendations_df = pd.concat(recommendations).drop_duplicates()
        recommendations_df = recommendations_df[~recommendations_df['parent_asin'].isin(reviewed_asins)]
        # 여기서 상위 5개로 결과 제한
        recommendations_df = recommendations_df.head(top_n)
        return recommendations_df
    else:
        return pd.DataFrame()


def get_recommendations_based_on_similarity(user_tfidf, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=5):
    cosine_sim_user = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    user_asins = user_reviews_df['parent_asin'].tolist()
    user_indices = [asin_to_index[asin] for asin in user_asins if asin in asin_to_index]
    sorted_indices = cosine_sim_user.argsort()[::-1]
    sorted_indices = [idx for idx in sorted_indices if idx not in user_indices]
    top_indices = sorted_indices[:top_n]
    recommended_products = category_df.iloc[top_indices]
    # 필요하다면 여기서도 head(top_n) 적용 가능 (이미 top_n 갯수로 제한되어 있으므로 불필요)
    return recommended_products.head(top_n)


In [24]:
# 5개 카테고리에 대해 반복 수행
for target_category in desired_categories:
    print("\n" + "="*50)
    print(f"Category: {target_category}")
    print("="*50)

    # 해당 카테고리만 필터링
    category_df = filtered_products_df[filtered_products_df['category'] == target_category].copy()
    category_df['combined_text'] = combine_text(category_df)
    user_reviews_df['combined_text'] = combine_text(user_reviews_df)

    # TF-IDF 계산
    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix = tfidf.fit_transform(category_df['combined_text'])

    category_df = category_df.reset_index(drop=True)
    category_df['index'] = category_df.index
    asin_to_index = pd.Series(category_df.index, index=category_df['parent_asin']).to_dict()

    print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

    high_rating_products_df = category_df[category_df['rating'] == rating_threshold].copy()
    print(f"Number of products with rating {rating_threshold} in {target_category}: {len(high_rating_products_df)}")

    # rating 기반 추천
    recommended_products_rating = get_recommendations_based_on_high_rating(user_reviews_df, high_rating_products_df, tfidf_matrix, asin_to_index, category_df, top_n=5)
    print("평가 +유사도  추천 상품:")
    print(recommended_products_rating[['parent_asin', 'product_title', 'category']])

    # 유사도 기반 추천
    user_query = ' '.join(user_reviews_df['combined_text'].tolist())
    user_tfidf = tfidf.transform([user_query])
    recommended_products_similarity = get_recommendations_based_on_similarity(user_tfidf, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=5)
    print("순수 유사도 기반 추천 상품:")
    print(recommended_products_similarity[['parent_asin', 'product_title', 'category']])


Category: Beauty
TF-IDF Matrix Shape: (9236, 5000)
Number of products with rating 5.0 in Beauty: 5437
평가 +유사도  추천 상품:
     parent_asin                                      product_title category
9048  B00FVHCR1M                 Denture Brush, 2 Count (Pack of 1)   Beauty
1511  B07RBKHKGV  150pcs 2 Inch Hair Clips No Slip Metal Hair Cl...   Beauty
8468  B071CK69W2  Women's Hair Bun Maker French Twist Hair Fold ...   Beauty
5004  B07N8HJGHW  doTERRA 2019 Essential Oil Cap Sticker Labels ...   Beauty
637   B003DLQNI6  Papier Poudre Oil Blotting Papers - Rachel 1 B...   Beauty
순수 유사도 기반 추천 상품:
     parent_asin                                      product_title category
2262  B09VB59RFG                            LOVE CINEMA Mascara 222   Beauty
7502  B00TXGZFHU  Beard/Stache Nutrients for Men: Beard Growth P...   Beauty
4759  B08JP8NKQT  Ombre Blonde Yellow Afro Curly Synthetic Wig w...   Beauty
7503  B00O965C0O  Natural 4D Silk Fiber Lash Mascara, Lengthenin...   Beauty
6533  B077D4921Z 